# SPR 2026 - Sentence Transformers

Embeddings pré-treinados + classificadores ML.

**Modelo:** paraphrase-multilingual-MiniLM-L12-v2

**Formato:** Code Competition (Kaggle) / Google Colab

In [ ]:
# ============================================================
# SETUP - Ambiente, Download de Dados e Google Drive
# ============================================================
import os
import sys

IS_KAGGLE = os.path.exists('/kaggle/input')
IS_COLAB = 'google.colab' in sys.modules

print(f"Ambiente: {'Kaggle' if IS_KAGGLE else 'Colab' if IS_COLAB else 'Local'}")

if IS_KAGGLE:
    DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
    OUTPUT_DIR = '/kaggle/working'
elif IS_COLAB:
    # Montar Google Drive para salvar outputs
    from google.colab import drive
    drive.mount('/content/drive')
    
    DRIVE_OUTPUT = '/content/drive/MyDrive/SPR_2026_outputs'
    os.makedirs(DRIVE_OUTPUT, exist_ok=True)
    OUTPUT_DIR = DRIVE_OUTPUT
    
    # Download dados Kaggle
    !pip install kaggle sentence-transformers lightgbm -q
    from google.colab import userdata
    os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
    !mkdir -p ~/.kaggle
    !echo '{"username":"'$KAGGLE_USERNAME'","key":"'$KAGGLE_KEY'"}' > ~/.kaggle/kaggle.json
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c spr-2026-mammography-report-classification -q
    !mkdir -p data && unzip -o -q spr-2026-mammography-report-classification.zip -d data/
    DATA_DIR = 'data'
else:
    DATA_DIR = '../data'
    OUTPUT_DIR = '../submissions'
    os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"DATA_DIR: {DATA_DIR}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgb
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

SEED = 42
N_FOLDS = 5
MODEL_NAME = 'paraphrase-multilingual-MiniLM-L12-v2'

np.random.seed(SEED)

## 1. Carregar Dados

In [ ]:
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(f"Train: {train.shape}")

test_path = os.path.join(DATA_DIR, 'test.csv')
if os.path.exists(test_path):
    test = pd.read_csv(test_path)
    print(f"Test: {test.shape}")
else:
    test = None
    print("test.csv não disponível - será carregado no runtime Kaggle")

## 2. Gerar Embeddings

In [ ]:
model = SentenceTransformer(MODEL_NAME)
print(f"Model loaded: {MODEL_NAME}")

print("Gerando embeddings do treino...")
X = model.encode(train['report'].tolist(), show_progress_bar=True, batch_size=32)
y = train['target'].values

print(f"X shape: {X.shape}")

In [ ]:
if test is not None:
    print("Gerando embeddings do teste...")
    X_test = model.encode(test['report'].tolist(), show_progress_bar=True, batch_size=32)
    print(f"X_test shape: {X_test.shape}")

## 3. Treinar Classificadores

In [ ]:
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED),
    'SVM': SVC(kernel='rbf', class_weight='balanced', random_state=SEED),
    'LightGBM': lgb.LGBMClassifier(n_estimators=200, max_depth=10, class_weight='balanced', random_state=SEED, verbose=-1),
}

results = {}
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)

for name, clf in models.items():
    print(f"Training {name}...")
    scores = cross_val_score(clf, X, y, cv=skf, scoring='f1_macro', n_jobs=-1)
    results[name] = scores
    print(f"  F1-Macro: {scores.mean():.4f} (+/- {scores.std()*2:.4f})")

best_name = max(results, key=lambda k: results[k].mean())
print(f"\nMelhor: {best_name}")

## 4. Gerar Submissão

In [ ]:
best_clf = models[best_name]
best_clf.fit(X, y)

if test is not None:
    predictions = best_clf.predict(X_test)
    
    submission = pd.DataFrame({'ID': test['ID'], 'target': predictions})
    submission.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)
    print(f"Submissão salva!")
    print(submission['target'].value_counts().sort_index())
else:
    print("Test não disponível - submeta no Kaggle")

In [ ]:
if IS_COLAB and test is not None:
    from google.colab import files
    files.download('submission.csv')